In [40]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [41]:
load_dotenv(override=True)

True

In [6]:
name = 'Benny'
title = 'LLM Engineer'
skills = 'AI Agents'

In [7]:
instructions1 = f"You are a job-seeking professional named {name}, applying for roles as a {title}. \
You are reaching out to an HR representative to introduce yourself. \
You highlight your strengths in {skills}, and write polite, professional outreach emails that are easy to understand and well-formatted."

instructions2 = f"You are a technically strong candidate named {name}, currently pursuing opportunities as a {title}. \
You are writing directly to a hiring manager, emphasizing your hands-on experience with {skills}. \
You write clear, confident, technically relevant messages that convey both competence and motivation."

instructions3 = f"You are a job-seeking professional named {name}, applying for roles as a {title}. \
You are introducing yourself to someone who may be connected to the hiring process, though their role is unclear. \
You maintain a neutral, respectful, and warm tone. You briefly mention your experience with {skills} and express openness to connect or discuss opportunities."

In [11]:
writer_agent1 = Agent(
    name="Professional Outreach Agent",
    instructions=instructions1,
    model="gpt-4.1-nano"
)

writer_agent2 = Agent(
    name="Technical Hiring Manager Agent",
    instructions=instructions2,
    model="gpt-4.1-nano"
)

writer_agent3 = Agent(
    name="General Introduction Agent",
    instructions=instructions3,
    model="gpt-4.1-nano"
)


In [9]:

result = Runner.run_streamed(writer_agent1, input="Write a introduction email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Introduction and Interest in LLM Engineer Position

Dear [Hiring Manager's Name],

I hope this message finds you well. My name is Benny, and I am reaching out to express my interest in the LLM Engineer position at [Company Name].

With a strong background in developing and deploying AI agents, I am passionate about advancing language model applications and creating intelligent solutions that enhance user experience. I am confident that my skills in natural language processing, model fine-tuning, and integration would be a valuable addition to your team.

I would be delighted to discuss how my experience aligns with your needs and explore potential opportunities to contribute to [Company Name]. Thank you very much for considering my application. I look forward to the possibility of working together.

Warm regards,  
Benny  
[Your LinkedIn profile or contact information]

In [ ]:
message = "Write a introduction email"

with trace("Parallel emails to hiring managers"):
    results = await asyncio.gather(
        Runner.run(writer_agent1, message),
        Runner.run(writer_agent2, message),
        Runner.run(writer_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Introducing Benny – Interested in LLM Engineer Role

Dear [Hiring Manager’s Name],  

I hope this message finds you well. My name is Benny, and I am excited to express my interest in the LLM Engineer position at [Company Name]. With a strong background in developing and deploying AI Agents, I am confident in my ability to contribute effectively to your team’s innovative projects.

Throughout my experience, I have specialized in designing sophisticated language models and integrating them into practical applications. I am passionate about advancing natural language understanding and creating systems that are intelligent, reliable, and user-friendly.

I would love the opportunity to discuss how my skills and background can align with your team’s goals. Thank you very much for considering my application. I look forward to the possibility of collaborating with you.

Warm regards,  
Benny  
[Your Contact Information]  
[LinkedIn Profile or Portfolio, if applicable]


Subject: Enthu

In [16]:
writer_picker = Agent(
    name="writer_picker",
    instructions="You are evaluating self-introduction emails. \
You are given multiple full email drafts (including subject and body). \
Pick the one you are most likely to respond to if you were a hiring manager or recruiter. \
Reply with the entire selected email content (subject and full body), and nothing else. \
Do not summarize. Do not explain. Just return the selected email verbatim.",
    model="gpt-4.1-nano"
)


In [17]:
message = "Write a introduction email"

with trace("Selection from writers"):
    results = await asyncio.gather(
        Runner.run(writer_agent1, message),
        Runner.run(writer_agent2, message),
        Runner.run(writer_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Introduction emails:\n\n".join(outputs)

    best = await Runner.run(writer_picker, emails)

    print(f"Best introduction email:\n{best.final_output}")


Best introduction email:
Subject: Enthusiastic LLM Engineer with Hands-on AI Agent Experience

Dear [Hiring Manager's Name],

I hope this message finds you well. My name is Benny, and I am currently pursuing opportunities as an LLM Engineer. With a strong technical background and hands-on experience developing and deploying AI Agents, I am excited about the prospect of contributing to your team.

In my recent projects, I’ve worked extensively with large language models, fine-tuning them for specialized tasks, and integrating them into multi-agent systems to improve robustness and contextual understanding. I am passionate about pushing the boundaries of what AI agents can achieve and am eager to bring my expertise in model development, system architecture, and problem-solving to [Company Name].

I would love the opportunity to discuss how my skills and experiences align with your team's goals. Thank you for considering my application. I look forward to the possibility of connecting.

Be

In [18]:
writer_agent1 = Agent(
    name="Professional Outreach Agent",
    instructions=instructions1,
    model="gpt-4.1-nano"
)

writer_agent2 = Agent(
    name="Technical Hiring Manager Agent",
    instructions=instructions2,
    model="gpt-4.1-nano"
)

writer_agent3 = Agent(
    name="General Introduction Agent",
    instructions=instructions3,
    model="gpt-4.1-nano"
)

In [20]:
writer_agent1

Agent(name='Professional Outreach Agent', instructions='You are a job-seeking professional named Benny, applying for roles as a LLM Engineer. You are reaching out to an HR representative to introduce yourself. You highlight your strengths in AI Agents, and write polite, professional outreach emails that are easy to understand and well-formatted.', handoff_description=None, handoffs=[], model='gpt-4.1-nano', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [42]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all target recipients """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("benny.guan97@gmail.com")  
    to_email = To("benny.guan97@gmail.com")  
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "writer email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [29]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all target recipients', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7C2B380>, strict_json_schema=True)

In [30]:
tool1 = writer_agent1.as_tool(tool_name="writer_agent1", tool_description="Write a Introduction email")
tool1

FunctionTool(name='writer_agent1', description='Write a Introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'writer_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7C2B920>, strict_json_schema=True)

In [43]:
description = "Write a introduction email"

tool1 = writer_agent1.as_tool(tool_name="writer_agent1", tool_description=description)
tool2 = writer_agent2.as_tool(tool_name="writer_agent2", tool_description=description)
tool3 = writer_agent3.as_tool(tool_name="writer_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='writer_agent1', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'writer_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7C2B9C0>, strict_json_schema=True),
 FunctionTool(name='writer_agent2', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'writer_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7CF6CA0>, strict_json_schema=True),
 FunctionTool(name='writer_agent3', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input

In [44]:
instructions = "You are a writing manager assisting job applicants. \
You do not write introduction emails yourself; you always use the tools provided. \
You try all 3 writer_agent tools once before choosing the best one. \
You pick the single best introduction email and use the send_email tool to send that email (and only that email) to the user."


writer_manager = Agent(
    name="Writer Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)
message = "Write an introduction email to a hiring manager"

with trace("Writer manager"):
    result = await Runner.run(writer_manager, message)


In [36]:
print(result)

RunResult:
- Last agent: Agent(name="Writer Manager", ...)
- Final output (str):
    It seems there was an issue sending the email. You can copy the email below and send it from your own email account.
    
    ---
    
    **Subject:** Introduction and Application for LLM Engineer Position
    
    Dear [Hiring Manager's Name],
    
    I hope this message finds you well. My name is Benny, and I am excited to introduce myself as a candidate for the LLM Engineer position at [Company Name]. With a strong background in AI agents and large language model development, I am passionate about pushing the boundaries of natural language understanding and generation.
    
    I am particularly drawn to [Company Name] because of its innovative approach to AI solutions and its commitment to advancing intelligent systems. I am eager to contribute my skills in developing and fine-tuning large language models, designing autonomous AI agents, and improving conversational capabilities to support your t

In [47]:
subject_instructions = "You are an email subject writer. \
You receive a message and write a subject line that is likely to get a response. \
Once you’re done, hand off control to the HTML email converter."

subject_writer = Agent(
    name="Email Subject Writer",
    instructions=subject_instructions,
    model="gpt-4.1-nano"
)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a introduction email")

In [48]:
html_instructions = "You are an HTML email body converter. \
You receive a subject and message text and convert them into a polished HTML email body. \
You are the last agent in the chain, so return the final email output to the user."

html_converter = Agent(
    name="HTML Email Converter",
    instructions=html_instructions,
    model="gpt-4.1-nano"
)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [49]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("benny.guan97@gmail.com")  
    to_email = To("benny.guan97@gmail.com")  
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [50]:
tools = [subject_tool, html_tool, send_html_email]

In [51]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7CF7C40>, strict_json_schema=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D5B149A0>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_jso

In [52]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [53]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='writer_agent1', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'writer_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7C2B9C0>, strict_json_schema=True), FunctionTool(name='writer_agent2', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'writer_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C6D7CF6CA0>, strict_json_schema=True), FunctionTool(name='writer_agent3', description='Write a introduction email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']

In [54]:
# 🎯 Writer Manager Instructions
writer_manager_instructions = "You are a writer manager helping job seekers craft great introduction emails. \
You never write emails yourself; you always use the tools provided. \
You try all 3 writer_agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best introduction email using your judgment of what will be most effective. \
After selecting the email, you hand off to the Email Sender agent to format and send the email."

#  Writer Manager Agent
writer_manager = Agent(
    name="Writer Manager",
    instructions=writer_manager_instructions,
    tools=tools,       # assuming tools = [writer_agent1, writer_agent2, writer_agent3]
    handoffs=handoffs, # assuming handoffs = [email_sender]
    model="gpt-4.1-nano"
)

email_sender_instructions = "You are an email sender. \
You receive a finalized introduction email from the writer manager and your job is to format it nicely and send it using the send_email tool."



message = "Write an introduction email to a hiring manager on behalf of Benny, a machine learning engineer."

with trace("Automated Intro Agent Flow"):
    result = await Runner.run(writer_manager, message)
